### Optimización de hiperparámetros (Optuna)

<a target="_blank" href="https://colab.research.google.com/github/pglez82/DeepLearningWeb/blob/master/labs/notebooks/Optimizaci%C3%B3n%20de%20hiperparámetros%20(Optuna).ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

En el entrenamiento de redes neuronales profundas, existen una multitud de hiperparámetros que podemos optimizar. Algunos de los más importantes son los siguientes:
- Learning rate
- Momento
- Tamaño del mini-batch
- Weight decay
- Cantidad de dropout
- Optimizador utilizado
- Número de capas de la red
- Tamaño de las capas de la red (u otros parámetros de la capa como tamaño del kernel para CNNs, etc)
- Funciones de activación usadas
- etc.

Es muy común que la búsqueda de hiperparámetros sea hecha de manera bastante artesanal, siguiendo la intuición y los conocimientos del científico de datos. Aún así, existe software que nos permite hacer esta búsqueda de manera más **sistemática y automática**. Uno de estos software es Optuna, que es el que veremos en esta práctica. Wandb también tiene su sistema de búsqueda de hiperparámetros.


### Instalación de los paquetes necesarios
Para esta práctica necesitaremos instalar Optuna.

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00


### Definición de la red

Para este ejemplo vamos a partir de la red de una práctica anterior (la usada para el conjunto Fashion MNIST).

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, dropout=0.2, linear_sizes = (50, 50, 10)):
        super(Net, self).__init__()
        self.layers = nn.Sequential()
        previous_size = 28*28 # la entrada tienen que coincidir con el número de pixeles en la imagen
        for i, linear_size in enumerate(linear_sizes):
            self.layers.append(nn.Linear(previous_size, linear_size))
            if i != len(linear_sizes):
                # Añadir dropout salvo en la última capa de salida
                self.layers.append(nn.Dropout(dropout))
            previous_size = linear_size

    def forward(self, x):
        x = x.view(-1, 28*28)
        return self.layers(x)

Como puedes ver, el constructor de esta red ya nos permite alterar el dropout y el número y tamaño de capas lineales sin tener que modificar nada. Esto va a ser muy útil para el uso de optuna.

### Carga de datos y creación de los dataloders

Optuna va a tratar de buscar los mejores hiperparámetros tratando de optimizar una métrica concreta. Esto dependerá del problema en particular. En este caso trataremos de **optimizar el acierto sobre el conjunto de validación**.

In [5]:
import torch
from torch.utils.data import random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


training_data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Utilizando dispositivo: %s" % device)

print("Datos de entrenamiento:")
print(training_data, end='\n\n')

# Separación de un conjunto de validación
training_data, validation_data = random_split(training_data,(50000,10000))

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, num_workers=2)
val_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True, num_workers=2)

Utilizando dispositivo: cpu
Datos de entrenamiento:
Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()



### Definición de los bucles de entrenamiento y validación
Estos bucles deben estar parametrizados para que cada entrenamiento use los hiperparámetros generados por Optuna

In [15]:
def validation(model, loss_module, val_dataloader):
    val_loss=0
    with torch.no_grad():
        model.eval()
        for data_inputs, data_labels in val_dataloader:
            data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)
            logits = model(data_inputs)
            val_loss += loss_module(logits, data_labels)
        return val_loss/ len(val_dataloader)

def train(train_dataloader, val_dataloader, dropout, linear_sizes, learning_rate, epoch_callback):
    # Creamos la red con los parámetros indicados por Optuna
    model = Net(dropout = dropout, linear_sizes = linear_sizes)

    # Definimos la función de pérdida
    loss_module = nn.CrossEntropyLoss()

    # Definimos el optimizador
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    min_loss = float('inf')
    patience = 3
    no_improvement = 0
    # Training loop
    for epoch in range(50):
        model.train()
        epoch_loss = 0
        for data_inputs, data_labels in train_dataloader:
            #Hacer una pasada hacia delante
            data_inputs = data_inputs.to(device)
            data_labels = data_labels.to(device)
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1)  # Output is [Batch size, 1], but we want [Batch size]
            #Calcular el valor de la función de pérdida para este mini-batch
            loss = loss_module(preds, data_labels)
            #Acumular el error (solo para luego mostrarlo)
            epoch_loss += loss.item()
            #Reiniciar los gradientes
            optimizer.zero_grad()
            #Pasada hacia atrás
            loss.backward()
            #Actualizar los parámetros
            optimizer.step()
        val_loss = validation(model, loss_module, val_dataloader)
        epoch_callback(val_loss, epoch)
        print("[Epoch %d] Training Loss %0.2f. Validation Loss %0.2f. Patience: %d/%d" % (epoch, epoch_loss/len(train_dataloader), val_loss, no_improvement, patience))
        if val_loss < min_loss:
            min_loss = val_loss
            no_improvement=0
        else:
            no_improvement += 1

        # parada temprana
        if no_improvement>=patience:
            return min_loss
    return min_loss


### Creación de la función objetivo
La función objetivo es la función que Optuna va a ejecutar en cada intento. En estos intentos Optuna calculará un conjunto de hiperparámetros basado en los hiperparámetros usados en intentos anteriores. Ten en cuenta que podríamos hacer una búsqueda exhaustiva de parámetros (estilo a un grid search), pero lo normal es hacer una búsqueda con algún tipo de heurístico que facilite la búsqueda de los mejores hiperparámetros. Luego hablaremos más de este tema.

In [16]:
import optuna

def objective(trial):
    # learning rate
    lr = trial.suggest_float("lr", 0.00001, 0.01, log=True)

    # Número y tamaño de las capas lineales
    num_linear_layers = trial.suggest_int("number_of_layers", 1, 3)
    linear_sizes=[]
    for i in range(num_linear_layers):
        linear_sizes.append(trial.suggest_int("linear_sizes{}".format(i), 1, 100))

    dropout = trial.suggest_float("dropout", 0, 0.5)
    batch_size = trial.suggest_int("batch_size", 2, 64)

    parameters = {'lr': lr, 'num_linear_layers':num_linear_layers, 'linear_sizes': linear_sizes, 'dropout':dropout, 'batch_size':batch_size}

    print("Empezando un nuevo intento con los siguientes parámetros:",parameters)

    #Esta función se llama al final de cada época y sirve para podar las ejecuciones menos prometedoras
    def epoch_callback(loss, epoch):
        trial.report(loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return train(train_dataloader, val_dataloader, dropout, linear_sizes, lr, epoch_callback)


### Lanzamiento de la búsqueda de hiperparámetros

In [ ]:
from optuna.pruners import HyperbandPruner
from optuna.trial import TrialState

pruner = HyperbandPruner()
study = optuna.create_study(
    direction="minimize",
    study_name="Fashion_mnist",
    storage="sqlite:///busqueda_hiperparametros.db",
    load_if_exists=True,
    pruner=pruner,
)
study.optimize(objective, n_trials=10)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-07-14 10:14:57,771] Using an existing study with name 'Fashion_mnist' instead of creating a new one.


Empezando un nuevo intento con los siguientes parámetros: {'lr': 0.003179050447220693, 'num_linear_layers': 1, 'linear_sizes': [57], 'dropout': 0.3306677276296845, 'batch_size': 18}
[Epoch 0] Training Loss 2.40. Validation Loss 1.55. Patience: 0/3
[Epoch 1] Training Loss 1.96. Validation Loss 1.27. Patience: 0/3
[Epoch 2] Training Loss 1.85. Validation Loss 1.13. Patience: 0/3
[Epoch 3] Training Loss 1.77. Validation Loss 1.04. Patience: 0/3
[Epoch 4] Training Loss 1.73. Validation Loss 0.97. Patience: 0/3
[Epoch 5] Training Loss 1.70. Validation Loss 0.92. Patience: 0/3
[Epoch 6] Training Loss 1.67. Validation Loss 0.88. Patience: 0/3
[Epoch 7] Training Loss 1.67. Validation Loss 0.85. Patience: 0/3
[Epoch 8] Training Loss 1.64. Validation Loss 0.82. Patience: 0/3
[Epoch 9] Training Loss 1.62. Validation Loss 0.80. Patience: 0/3
[Epoch 10] Training Loss 1.62. Validation Loss 0.78. Patience: 0/3
[Epoch 11] Training Loss 1.59. Validation Loss 0.76. Patience: 0/3
[Epoch 12] Training Loss

[I 2023-07-14 10:21:43,690] Trial 3 finished with value: 0.5617775321006775 and parameters: {'batch_size': 18, 'dropout': 0.3306677276296845, 'linear_sizes0': 57, 'lr': 0.003179050447220693, 'number_of_layers': 1}. Best is trial 3 with value: 0.5617775321006775.


[Epoch 49] Training Loss 1.48. Validation Loss 0.56. Patience: 0/3
Empezando un nuevo intento con los siguientes parámetros: {'lr': 0.0005563731383704818, 'num_linear_layers': 1, 'linear_sizes': [19], 'dropout': 0.2198276672290051, 'batch_size': 6}
[Epoch 0] Training Loss 2.35. Validation Loss 1.90. Patience: 0/3
[Epoch 1] Training Loss 1.84. Validation Loss 1.56. Patience: 0/3
[Epoch 2] Training Loss 1.65. Validation Loss 1.39. Patience: 0/3
[Epoch 3] Training Loss 1.53. Validation Loss 1.28. Patience: 0/3
[Epoch 4] Training Loss 1.46. Validation Loss 1.20. Patience: 0/3
[Epoch 5] Training Loss 1.41. Validation Loss 1.14. Patience: 0/3
[Epoch 6] Training Loss 1.37. Validation Loss 1.09. Patience: 0/3
[Epoch 7] Training Loss 1.33. Validation Loss 1.05. Patience: 0/3
[Epoch 8] Training Loss 1.30. Validation Loss 1.02. Patience: 0/3
[Epoch 9] Training Loss 1.29. Validation Loss 0.99. Patience: 0/3


### Monitorización del proceso
Una herramienta muy útil para ver el proceso de Optuna es optuna-dashboard. Esta herramienta abre un interfaz web donde podemos ver el progreso de la búsqueda de hiperparámetros. Para lanzarla, necesitamos instalarla y dar acceso a la misma a la base de datos donde se guarda la información relativa a la búsqueda. En Google Colab, puede hacerse de esta manera:

In [18]:
!pip install optuna-dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 12.2 MB/s eta 0:00:00


In [21]:
    import time
    import threading
    from optuna_dashboard import wsgi
    import optuna
    from wsgiref.simple_server import make_server


    port = 1234
    storage = optuna.storages.RDBStorage("sqlite:////content/busqueda_hiperparametros.db")
    app = wsgi(storage)
    httpd = make_server("localhost", port, app)
    thread = threading.Thread(target=httpd.serve_forever)
    thread.start()
    time.sleep(3) # Wait until the server startup

    from google.colab import output
    output.serve_kernel_port_as_window(port, path='/dashboard/')

<IPython.core.display.Javascript object>